In [1]:
# Imports 

from helper import ClassConstructor
from db.players_controller import PlayersController
from db.clubs_controller import ClubsController
from db.stadiums_controller import StadiumsController
from db.games_controller import GamesController

from logs_helper import LogsHandler

# Just int case
from pprint import pprint

In [2]:
# Class declare (statics)

class_const = ClassConstructor()
players_controller = PlayersController()
clubs_controller = ClubsController()
stadiums_controller = StadiumsController()
games_controller = GamesController()

logs_handler = LogsHandler()

season = '2022'

In [3]:
# Select and transform stadium data into objects

stadiums_data = stadiums_controller.select_all_stadiums()
stadiums = class_const.stadiums(stadiums_data)

In [4]:
# Select players with contract

# get from db
players_data = players_controller.select_players_with_contract(season) 

In [5]:
# Serie A clubs selection & cast

serie_a = clubs_controller.select_serie_a_clubs(season)
serie_a_clubs = class_const.prepare_clubs(serie_a, players_data)

In [6]:
# Confirm that clubs are created

serie_a_clubs

[Club(América Mineiro),
 Club(Atlético Mineiro),
 Club(Athletico Paranaense),
 Club(Ceará),
 Club(Fortaleza),
 Club(Flamengo),
 Club(Fluminense),
 Club(Cuiabá),
 Club(Grêmio),
 Club(International),
 Club(Chapecoense),
 Club(Juventude),
 Club(São Paulo),
 Club(Corinthians),
 Club(Palmeiras),
 Club(Santos),
 Club(Red Bull Bragantino),
 Club(Sport Recife),
 Club(Bahia),
 Club(Atlético Goianiense)]

In [7]:
# Check for start eleven 

serie_a_clubs[0].start_eleven

[Player(Esteban Viramontez),
 Player(Kara Lampião),
 Player(Yardley da Costa),
 Player(Francisco Nochez),
 Player(Rodrigo Hafez),
 Player(Quiromeu Matheus),
 Player(Kentaro Celso),
 Player(Angélico da Silva),
 Player(Benicio Gonalez),
 Player(Antero Pera),
 Player(Alfeno Aguiar)]

In [8]:
# Define the clubs matches, this will return a list of lists with two differents teams

serie_a_schedule = class_const.define_schedule(serie_a_clubs)

In [9]:
# Return a list of games object with all the confrontations through the season

serie_a_games = class_const.prepare_games(serie_a_schedule, stadiums, 'Campeonato Brasileiro Serie A', 1, int(season))

In [10]:
serie_a_games[0].start()

{'game_stats': {'América Mineiro': {'home': True,
   'goals': 0,
   'shots': 6,
   'shots on target': 4,
   'fouls': 15,
   'passes': 81,
   'wrong passes': 23,
   'interceptions': 20,
   'tackles': 12,
   'stolen_balls': 16,
   'saves': 12,
   'ball possession': 0,
   'offsides': 0,
   'free kicks': 0,
   'penalties': 0},
  'Atlético Mineiro': {'home': True,
   'goals': 1,
   'shots': 4,
   'shots on target': 13,
   'fouls': 9,
   'passes': 71,
   'wrong passes': 19,
   'interceptions': 18,
   'tackles': 16,
   'stolen_balls': 16,
   'saves': 4,
   'ball possession': 0,
   'offsides': 0,
   'free kicks': 0,
   'penalties': 0}},
 'scoreboard': '0 x 1',
 'player_stats': {'assists': defaultdict(int, {Player(Hernán Garmendez): 1}),
  'goals': defaultdict(int, {Player(Javier Herandez): 1}),
  'tackles': defaultdict(int,
              {Player(Estefano Albarez): 5,
               Player(Benicio Gonalez): 1,
               Player(Javier Herandez): 4,
               Player(Pelasco Brito): 5,
 

In [11]:
"""
TODO

adicionar mais uma partida a todos os jogadores que entraram na partida

adicionar clean_sheets aos defensores caso nao tenha rolado gols
adicionar goals_conceded aos defensores caso tenham sofrido gols


adicionar clearances ao algoritimo

melhorar o algoritimo para que tenha mais passes, desarmes e tackles durante as partidas
"""

'\nTODO\n\nadicionar mais uma partida a todos os jogadores que entraram na partida\n\nadicionar clean_sheets aos defensores caso nao tenha rolado gols\nadicionar goals_conceded aos defensores caso tenham sofrido gols\n\n\nadicionar clearances ao algoritimo\n\nmelhorar o algoritimo para que tenha mais passes, desarmes e tackles durante as partidas\n'

In [12]:
logs = serie_a_games[0].logs
game = serie_a_games[0]

In [13]:
stats_data = logs_handler.prepare_game_stats_logs_to_db(logs_handler.get_game_stats(logs, game.home, game.away))

In [14]:
# Check for the stats data

stats_data

[[0, 6, 4, 15, 81, 23, 20, 12, 16, 12, 0, 0, 0, 0, 1],
 [1, 4, 13, 9, 71, 19, 18, 16, 16, 4, 0, 0, 0, 0, 2]]

In [15]:
# Insert game stats into database

s1 = games_controller.insert_game_stat_with_id_return(stats_data[0])
s2 = games_controller.insert_game_stat_with_id_return(stats_data[1])

In [16]:
# Get game stats ids

game_stats_ids = s1[0][0], s2[0][0]

In [17]:
# Prepare game data

prepare_game = logs_handler.prepare_game_logs_to_db(game.logs, game_stats_ids)

In [18]:
# Get game id

game_id = games_controller.insert_game(prepare_game)

In [19]:
# After this is the preparation to insert player stats per game on databse

game_id = game_id[0][0]

In [20]:
player_stats = game.logs['stats']

In [32]:
player_stats['Hernán Garmendez']

{'matches': 1,
 'shots': 1,
 'shots_on_target': 0,
 'goals': 0,
 'assists': 1,
 'fouls_committed': 0,
 'tackles': 1,
 'passes': 2,
 'wrong_passes': 1,
 'intercepted_passes': 0,
 'clearances': 0,
 'stolen_balls': 0,
 'clean_sheets': 1,
 'defenses': 0,
 'difficult_defenses': 0,
 'goals_conceded': 0,
 'player_id': 43}

In [29]:
game.away.start_eleven

[Player(Anselmo Soares),
 Player(Idário Aguiar),
 Player(Piero Urquidez),
 Player(Jin Teles),
 Player(Paquetá Cella),
 Player(Estefano Albarez),
 Player(Javier Herandez),
 Player(Raban Viveiro),
 Player(Lautaro Maidana),
 Player(Hernán Garmendez),
 Player(Alejandro Martinez)]

In [23]:
from data_helper import prepare_player_stats_to_db

In [24]:
player_stats_data = prepare_player_stats_to_db(player_stats, season, game_id)

In [25]:
player_stats_data

[['2022', 1, 0, 0, 0, 0, 0, 0, 16, 6, 0, 0, 0, 0, 5, 7, 1, 1, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 5709],
 ['2022', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 5709],
 ['2022', 1, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 7, 5709],
 ['2022', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 8, 5709],
 ['2022', 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 9, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 5709],
 ['2022', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 12, 5709],
 ['2022', 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 13, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 5709],
 ['2022', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [26]:
players_controller.insert_players_stats(player_stats_data)

In [27]:
# Prepare a stats data for game_stats 
"""
for game in serie_a_games:
    stats_data = logs_handler.prepare_game_stats_logs_to_db(logs_handler.get_game_stats(game.logs, game.home, game.away))

    # Insert and get the id from the game stats
    s1 = games_controller.insert_game_stat_with_id_return(stats_data[0])
    s2 = games_controller.insert_game_stat_with_id_return(stats_data[1])

    # get the last games_stats id inserted
    game_ids = s1[0][0], s2[0][0]

    # prepare data to insert game
    prepare_game = logs_handler.prepare_game_logs_to_db(game.logs, game_ids)

    # insert games into the database
    games_controller.insert_games_list([prepare_game])
"""

'\nfor game in serie_a_games:\n    stats_data = logs_handler.prepare_game_stats_logs_to_db(logs_handler.get_game_stats(game.logs, game.home, game.away))\n\n    # Insert and get the id from the game stats\n    s1 = games_controller.insert_game_stat_with_id_return(stats_data[0])\n    s2 = games_controller.insert_game_stat_with_id_return(stats_data[1])\n\n    # get the last games_stats id inserted\n    game_ids = s1[0][0], s2[0][0]\n\n    # prepare data to insert game\n    prepare_game = logs_handler.prepare_game_logs_to_db(game.logs, game_ids)\n\n    # insert games into the database\n    games_controller.insert_games_list([prepare_game])\n'

In [28]:
# Ajeitar o algoritimo pq goleiro ta chutando no alvo e no gol 
# 
#